In [187]:
import numpy as np
import pandas as pd

In [188]:
df1 = pd.read_csv("modeling data/1_players_info_added_Clean_data.csv")
df1_nulls = df1.isnull().sum()
print(df1_nulls)
print(len(df1))

Unnamed: 0           0
Location             0
Date                 0
Round                0
Winner               0
Loser                0
WRank               60
LRank              206
W_hand               0
W_ht               957
W_nationality      858
W_age              859
L_hand               0
L_ht              1102
L_nationality      791
L_age              796
L_wt             11818
W_wt             11801
dtype: int64
19922


In [189]:
df1_dropped_wt = df1.copy()
df1_dropped_wt.drop(columns=["Unnamed: 0",'L_wt', 'W_wt'], inplace=True)
df1_no_nulls = df1_dropped_wt.dropna()
len(df1_no_nulls)

17917

In [190]:
df_winner = df1_no_nulls.copy()
df_loser = df1_no_nulls.copy()

df_winner['win'] = 1
df_loser['win'] = 0

# Renaming columns to 'player' and 'opponent' for winners
df_winner.rename(columns={
    'Winner': 'player',
    'Loser': 'opponent',
    'WRank': 'player_rank',
    'LRank': 'opponent_rank',
    'W_hand': 'player_hand',
    'L_hand': 'opponent_hand',
    'W_ht': 'player_ht', 
    'W_nationality':'player_nationality', 
    'W_age':'player_age', 
    'L_ht':'opponent_ht',
    'L_nationality':'opponent_nationality', 
    'L_age':'opponent_age'
}, inplace=True)

# Renaming columns to 'player' and 'opponent' for losers
df_loser.rename(columns={
    'Winner': 'opponent',
    'Loser': 'player',
    'WRank': 'opponent_rank',
    'LRank': 'player_rank',
    'W_hand': 'opponent_hand',
    'L_hand': 'player_hand',
    'W_ht':'opponent_ht', 
    'W_nationality':'opponent_nationality', 
    'W_age':'opponent_age', 
    'L_ht':'player_ht',
    'L_nationality':'player_nationality',
    'L_age':'player_age'
}, inplace=True)
df_transformed = pd.concat([df_winner, df_loser], ignore_index=True)
# df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)

In [191]:
player_peak_ranking = df_transformed.groupby('player')['player_rank'].min().reset_index()
opponent_peak_ranking = df_transformed.groupby('opponent')['opponent_rank'].min().reset_index()
player_peak_ranking

player_updated = pd.merge(df_transformed, player_peak_ranking[['player', 'player_rank']], on='player', how='left')
opponent_updated = pd.merge(player_updated, opponent_peak_ranking[['opponent', 'opponent_rank']], on='opponent', how='left')
df1 = opponent_updated.copy()
df1.drop(columns=['player_rank_x', 'opponent_rank_x'], inplace=True)
df1 = df1.rename(columns={  'player_rank_y': 'player_rank',
                            'opponent_rank_y': 'opponent_rank'})
df1

,Location,Date,Round,player,opponent,player_hand,player_ht,player_nationality,player_age,opponent_hand,opponent_ht,opponent_nationality,opponent_age,win,player_rank,opponent_rank
0,Brisbane,1/1/2017,Round of 128,Thompson J.,Ymer E.,R,183.0,AUS,24.6,R,183.0,SWE,22.8,1,43.0,108.0
1,Brisbane,1/2/2017,Round of 128,Mahut N.,Robert S.,R,190.0,FRA,37.3,R,185.0,FRA,39.6,1,38.0,53.0
2,Brisbane,1/2/2017,Round of 128,Ferrer D.,Tomic B.,R,175.0,ESP,36.7,R,193.0,AUS,26.2,1,21.0,26.0
3,Brisbane,1/2/2017,Round of 128,Edmund K.,Escobedo E.,R,188.0,GBR,24.1,R,185.0,MEX,23.1,1,14.0,69.0
4,Brisbane,1/2/2017,Round of 128,Dimitrov G.,Johnson S.,R,188.0,BUL,27.6,R,188.0,USA,29.0,1,3.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,nitto-atp-finals,1/1/2017,Round Robin,Busta P.,Thiem D.,R,188.0,ESP,33.0,R,185.0,AUT,31.0,0,15.0,3.0
35830,nitto-atp-finals,1/1/2017,Round Robin,Cilic M.,Sock J.,R,198.0,CRO,36.0,R,191.0,USA,32.0,0,3.0,8.0
35831,nitto-atp-finals,1/1/2017,Round Robin,Busta P.,Dimitrov G.,R,188.0,ESP,33.0,R,191.0,BUL,33.0,0,15.0,3.0
35832,nitto-atp-finals,1/1/2017,Round Robin,Goffin D.,Dimitrov G.,R,180.0,BEL,34.0,R,191.0,BUL,33.0,0,7.0,3.0


In [192]:
countryList = [
    ["AUS", "Australia", "Canberra", "Sydney", "Melbourne", "Brisbane", "Perth", "Adelaide"],
    ["IND", "India", "New Delhi", "Mumbai", "Kolkata", "Chennai", "Bangalore", "Hyderabad", "Pune"],
    ["QAT", "Qatar", "Doha"],
    ["NZL", "New Zealand", "Wellington", "Auckland", "Christchurch"],
    ["FRA", "France", "Paris", "Marseille", "Lyon", "Toulouse", "Nice", "Nantes", "Strasbourg", "Montpellier", "Bordeaux", "Lille","Roland Garros","Metz"],
    ["ECU", "Ecuador", "Quito", "Guayaquil"],
    ["BGR", "Bulgaria", "Sofia", "Plovdiv", "Varna"],
    ["ARG", "Argentina", "Buenos Aires", "Cordoba", "Rosario", "Mendoza"],
    ["USA", "US", "United States", "New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "Salem", "Cincinnati","Newport",'Atlanta','Washington','Memphis',"Delray Beach","Miami"],
    ["BRA", "Brazil", "Brasília", "São Paulo", "Sao Paulo","Rio de Janeiro","Rio", "Salvador", "Fortaleza", "Belo Horizonte"],
    ["MEX", "Mexico", "Mexico City", "Guadalajara", "Monterrey", "Puebla", "Toluca", "Tijuana","Acapulco","Los Cabos"],
    ["ARE", "United Arab Emirates", "Abu Dhabi", "Dubai", "Sharjah"],
    ["MAR", "Morocco", "Rabat", "Casablanca", "Fez", "Marrakech"],
    ["MCO", "Monaco", "Monte Carlo"],
    ["ESP", "Spain", "Madrid", "Barcelona", "Valencia", "Seville", "Zaragoza"],
    ["HUN", "Hungary", "Budapest"],
    ["PRT", "Portugal", "Lisbon", "Porto", "Amadora", "Braga","Estoril"],
    ["TUR", "Turkey", "Ankara", "Istanbul", "Izmir", "Bursa", "Antalya"],
    ["DEU", "Germany", "Berlin", "Hamburg", "Munich", "Cologne", "Frankfurt", "Stuttgart", "Düsseldorf","Dusseldorf", "Dortmund", "Essen", "Leipzig","Halle"],
    ["ITA", "Italy", "Rome", "Milan", "Naples", "Turin", "Palermo", "Genoa", "Bologna", "Florence","nitto","Sardinia"],
    ["CHE", "Switzerland", "Zurich", "Geneva", "Basel", "Lausanne","Gstaad"],
    ["NLD", "Netherlands", "Amsterdam", "Rotterdam", "The Hague", "Utrecht","s Hertogenbosch"],
    ["GBR", "United Kingdom", "London", "Birmingham", "Glasgow", "Liverpool", "Leeds", "Sheffield", "Edinburgh", "Bristol", "Manchester","Eastbourne","wimbledon"],
    ["SWE", "Sweden", "Stockholm", "Gothenburg", "Malmö","Malmo", "Uppsala",'bastad'],
    ["HRV", "Croatia", "Zagreb", "Split", "Rijeka",'Umag'],
    ["CHN", "China", "Beijing", "Shanghai", "Chongqing", "Tianjin", "Guangzhou", "Shenzhen", "Chengdu","Zhuhai"],
    ["BEL", "Belgium", "Brussels", "Antwerp", "Ghent", "Charleroi", "Liège"],
    ["RUS", "Russia", "Moscow", "Saint Petersburg","st petersburg","Petersburg" "Novosibirsk", "Yekaterinburg", "Nizhny Novgorod"],
    ["CHL", "Chile", "Santiago"],
    ["CAN", "Canada", "Ottawa", "Toronto", "Montreal", "Calgary", "Edmonton", "Vancouver", "Winnipeg", "Quebec City", "Hamilton", "Kitchener"],
    ["KAZ", "Kazakhstan", "Nur Sultan", "Almaty"],
    ["AUT", "Austria", "Vienna","Kitzbuhel"],
    ["JPN", "Japan", "Tokyo", "Yokohama", "Osaka", "Nagoya", "Sapporo", "Fukuoka", "Kobe", "Kyoto"],
    ["Unknown", "Davis","Club",'Cup', 'Tour Finals',"NextGen"]
]

In [193]:
import re

newCountryName = df1.copy()
newCountryName["Location"] = newCountryName["Location"].str.replace('_'," ")
newCountryName["Location"] = newCountryName["Location"].str.replace('-'," ")

for country in countryList:
    for i in range(len(country)):
        newCountryName["Location"] = newCountryName["Location"].str.replace('.*'+country[i]+'.*', country[0], flags=re.IGNORECASE, regex=True)
newCountryName["Location"].unique()

array(['USA', 'IND', 'QAT', 'NZL', 'FRA', 'ECU', 'BGR', 'ARG', 'NLD',
       'BRA', 'MEX', 'ARE', 'MAR', 'MCO', 'ESP', 'HUN', 'PRT', 'TUR',
       'DEU', 'ITA', 'CHE', 'Unknown', 'GBR', 'SWE', 'HRV', 'AUT', 'CAN',
       'RUS', 'CHN', 'JPN', 'BEL', 'KAZ', 'CHL'], dtype=object)

In [195]:
df1_no_location = df1.copy()
df1_no_location.drop(columns=["Location",'player_nationality', 'opponent_nationality'], inplace=True)
df1_no_location

,Date,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank
0,1/1/2017,Round of 128,Thompson J.,Ymer E.,R,183.0,24.6,R,183.0,22.8,1,43.0,108.0
1,1/2/2017,Round of 128,Mahut N.,Robert S.,R,190.0,37.3,R,185.0,39.6,1,38.0,53.0
2,1/2/2017,Round of 128,Ferrer D.,Tomic B.,R,175.0,36.7,R,193.0,26.2,1,21.0,26.0
3,1/2/2017,Round of 128,Edmund K.,Escobedo E.,R,188.0,24.1,R,185.0,23.1,1,14.0,69.0
4,1/2/2017,Round of 128,Dimitrov G.,Johnson S.,R,188.0,27.6,R,188.0,29.0,1,3.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,1/1/2017,Round Robin,Busta P.,Thiem D.,R,188.0,33.0,R,185.0,31.0,0,15.0,3.0
35830,1/1/2017,Round Robin,Cilic M.,Sock J.,R,198.0,36.0,R,191.0,32.0,0,3.0,8.0
35831,1/1/2017,Round Robin,Busta P.,Dimitrov G.,R,188.0,33.0,R,191.0,33.0,0,15.0,3.0
35832,1/1/2017,Round Robin,Goffin D.,Dimitrov G.,R,180.0,34.0,R,191.0,33.0,0,7.0,3.0


In [196]:
df1_home_country = newCountryName.copy()
df1_home_country['is_player_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['player_nationality'])
df1_home_country['is_opponent_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['opponent_nationality'])
df1_home_country.drop(columns=["Location",'player_nationality', 'opponent_nationality'], inplace=True)
df1_home_country

,Date,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country
0,1/1/2017,Round of 128,Thompson J.,Ymer E.,R,183.0,24.6,R,183.0,22.8,1,43.0,108.0,False,False
1,1/2/2017,Round of 128,Mahut N.,Robert S.,R,190.0,37.3,R,185.0,39.6,1,38.0,53.0,False,False
2,1/2/2017,Round of 128,Ferrer D.,Tomic B.,R,175.0,36.7,R,193.0,26.2,1,21.0,26.0,False,False
3,1/2/2017,Round of 128,Edmund K.,Escobedo E.,R,188.0,24.1,R,185.0,23.1,1,14.0,69.0,False,False
4,1/2/2017,Round of 128,Dimitrov G.,Johnson S.,R,188.0,27.6,R,188.0,29.0,1,3.0,25.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,1/1/2017,Round Robin,Busta P.,Thiem D.,R,188.0,33.0,R,185.0,31.0,0,15.0,3.0,False,False
35830,1/1/2017,Round Robin,Cilic M.,Sock J.,R,198.0,36.0,R,191.0,32.0,0,3.0,8.0,False,False
35831,1/1/2017,Round Robin,Busta P.,Dimitrov G.,R,188.0,33.0,R,191.0,33.0,0,15.0,3.0,False,False
35832,1/1/2017,Round Robin,Goffin D.,Dimitrov G.,R,180.0,34.0,R,191.0,33.0,0,7.0,3.0,False,False


In [238]:
def split_date(df):
    df1 = df.copy()
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1['year'] = df1['Date'].dt.year
    df1['month'] = df1['Date'].dt.month
    df1['day'] = df1['Date'].dt.day
    df1['dayofweek'] = df1['Date'].dt.dayofweek
    df1.drop(columns=["Date"], inplace=True)
    return df1

In [239]:
split_date(df1_home_country).to_csv('modeling data/df1_with_home.csv', index=False)
split_date(df1_no_location).to_csv('modeling data/df1_without_home.csv', index=False)
df_home = pd.read_csv("modeling data/df1_with_home.csv")
df_no_home = pd.read_csv("modeling data/df1_without_home.csv")

In [240]:
#encoding the date using sin and cos to preserve the cyclical nature of
def encode_date(df):
    df_date_encoded = df_home.copy()
    df_date_encoded['month_sin'] = np.sin(2 * np.pi * df_date_encoded['month']/12)
    df_date_encoded['month_cos'] = np.cos(2 * np.pi * df_date_encoded['month']/12)
    df_date_encoded['dayofweek_sin'] = np.sin(2 * np.pi * df_date_encoded['dayofweek']/7)
    df_date_encoded['dayofweek_cos'] = np.cos(2 * np.pi * df_date_encoded['dayofweek']/7)
    df_date_encoded['day_sin'] = np.sin(2 * np.pi * df_date_encoded['day']/30.5)
    df_date_encoded['day_cos'] = np.cos(2 * np.pi * df_date_encoded['day']/30.5)
    df_date_encoded.drop(columns=["month","dayofweek", "day"], inplace=True)
    return df_date_encoded

In [241]:
#encoding rounds, no need to scale because the higher the round is, the more important it is
def encode_round(df):
    df_round_encoded = df.copy()
    round_mapping = {
        '1st Round Qualifying': 1,
        '2nd Round Qualifying': 2,
        '3rd Round Qualifying': 3,
        'Round Robin': 4,
        'Round of 128': 5,
        'Round of 64': 6,
        'Round of 32': 7,
        'Round of 16': 8,
        'Quarterfinals': 9,
        'Semifinals': 10,
        'The Final': 11
    }
    df_round_encoded['Round'] = df_round_encoded['Round'].map(round_mapping)
    return df_round_encoded

In [242]:
#binary encoding for columns in columnNames = ['is_player_home_country','is_opponent_home_country',"is_taller", "is_older", "rank_is_higher"]
def binary_column(df):
    df_binary = df.copy()
    df_binary['is_older'] = df_binary['player_age'] > df_binary['opponent_age']
    df_binary['is_taller'] = df_binary['player_ht'] > df_binary['opponent_ht']
    df_binary['rank_is_higher'] = df_binary['player_rank'] > df_binary['opponent_rank']

    df_binary.drop(columns=["player_age",'opponent_age', 'player_ht',"opponent_ht",'player_rank', 'opponent_rank'], inplace=True)
    return df_binary

In [243]:
#integer encoding for handedness
def int_encode_hand(df):
    df_hand_int_encoded = df.copy()
    hand_mapping = {
        'R': 1,
        'L': 2,
        'U': 0
    }
    df_hand_int_encoded['player_hand'] = df_hand_int_encoded['player_hand'].map(hand_mapping)
    df_hand_int_encoded['opponent_hand'] = df_hand_int_encoded['opponent_hand'].map(hand_mapping)
    return df_hand_int_encoded

In [244]:
#one-hot encoding for handedness
def hot_encode_hand(df):
    df_hand_1hot_encoded = df.copy()

    df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['player_hand'], prefix='player_hand')
    df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['opponent_hand'], prefix='opponent_hand')
    return df_hand_1hot_encoded

In [245]:
def encode_pipeline(df):
    df1 = df.copy()
    df2 = encode_date(df1)
    df3 = encode_round(df2)
    df_int_hand = int_encode_hand(df3)
    df_hot_hand = hot_encode_hand(df3)
    return df_int_hand,df_hot_hand

In [246]:
df_hand_int_encoded_home, df_hand_1hot_encoded_home = encode_pipeline(df_home)


df_hand_int_encoded_bool = binary_column(df_hand_int_encoded_home)
df_hand_1hot_encoded_bool = binary_column(df_hand_1hot_encoded_home)

In [247]:
#normalize function
numeric_columns_1 = ['player_ht', 'opponent_ht']
numeric_columns_2 = ['player_age','opponent_age']
numeric_columns_3 = ['player_rank', 'opponent_rank']
numeric_columns_4 = ["year"]

def normalize(og_df, numeric_columns):
    df = og_df.copy()
    for column in numeric_columns:
        # Z-score normalization
        df[f'{column}_zscore'] = (df[column] - df[column].mean()) / df[column].std()
        # Min-max scaling
        df[f'{column}_minmax'] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
        # Decimal scaling
        max_length = max(df[column].apply(lambda x: len(str(np.floor(x))) if x > 0 else 1))
        df[f'{column}_decimal'] = df[column] / (10**max_length)
    return df

In [249]:
#no normalization (NN) has home
df_hand_int_encoded_NN = df_hand_int_encoded_home.copy()
df_hand_1hot_encoded_NN = df_hand_1hot_encoded_home.copy()

#normalize all
df_hand_int_N_all = normalize(df_hand_int_encoded_NN,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
df_hand_1hot_N_all = normalize(df_hand_1hot_encoded_NN,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

df_hand_int_N_all.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)
df_hand_1hot_N_all.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)



#no normalization (NN) no home
df_hand_int_encoded_NN_bool = df_hand_int_encoded_bool.copy()
df_hand_1hot_encoded_NN_bool = df_hand_1hot_encoded_bool.copy()

#normalize all
df_hand_int_encoded_N_year_bool = normalize(df_hand_int_encoded_NN_bool,numeric_columns_4)
df_hand_1hot_encoded_N_year_bool = normalize(df_hand_1hot_encoded_NN_bool,numeric_columns_4)

df_hand_int_encoded_N_year_bool.drop(columns=numeric_columns_4, inplace=True)
df_hand_1hot_encoded_N_year_bool.drop(columns=numeric_columns_4, inplace=True)
df_hand_1hot_encoded_N_year_bool

,Round,player,opponent,win,is_player_home_country,is_opponent_home_country,month_sin,month_cos,dayofweek_sin,dayofweek_cos,...,player_hand_U,opponent_hand_L,opponent_hand_R,opponent_hand_U,is_older,is_taller,rank_is_higher,year_zscore,year_minmax,year_decimal
0,5,Thompson J.,Ymer E.,1,False,False,0.5,0.866025,-0.781831,0.62349,...,0,0,1,0,True,False,False,-0.535105,0.25,0.002017
1,5,Mahut N.,Robert S.,1,False,False,0.5,0.866025,0.000000,1.00000,...,0,0,1,0,False,True,False,-0.535105,0.25,0.002017
2,5,Ferrer D.,Tomic B.,1,False,False,0.5,0.866025,0.000000,1.00000,...,0,0,1,0,True,False,False,-0.535105,0.25,0.002017
3,5,Edmund K.,Escobedo E.,1,False,False,0.5,0.866025,0.000000,1.00000,...,0,0,1,0,True,True,False,-0.535105,0.25,0.002017
4,5,Dimitrov G.,Johnson S.,1,False,True,0.5,0.866025,0.000000,1.00000,...,0,0,1,0,False,False,False,-0.535105,0.25,0.002017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,4,Busta P.,Thiem D.,0,False,False,0.5,0.866025,-0.781831,0.62349,...,0,0,1,0,True,True,True,-0.535105,0.25,0.002017
35830,4,Cilic M.,Sock J.,0,False,False,0.5,0.866025,-0.781831,0.62349,...,0,0,1,0,True,True,False,-0.535105,0.25,0.002017
35831,4,Busta P.,Dimitrov G.,0,False,False,0.5,0.866025,-0.781831,0.62349,...,0,0,1,0,False,False,True,-0.535105,0.25,0.002017
35832,4,Goffin D.,Dimitrov G.,0,False,False,0.5,0.866025,-0.781831,0.62349,...,0,0,1,0,True,False,True,-0.535105,0.25,0.002017


In [250]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

#split df into training and testing
def split_df(df):
    working_df = df.copy()
    X = working_df.drop(['win', 'player', 'opponent'], axis=1)  # Exclude non-numeric or target columns
    y = working_df['win']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    return X_train, X_test, y_train, y_test

#train SVC model
def lin_svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    lsvm_model = LinearSVC(dual=True, max_iter=10000, random_state=1)
    lsvm_model.fit(X_train, y_train)
    y_pred_lsvm = lsvm_model.predict(X_test)
    print(classification_report(y_test, y_pred_lsvm))
    return y_test, y_pred_lsvm

#train SVC model
def svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    svm_model = SVC(kernel='linear', probability=True, random_state=1)
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    print(classification_report(y_test, y_pred_svm))
    return y_test, y_pred_svm, svm_model, X_test

#train random forest model
def rfc_model(X_train, X_test, y_train, y_test):
    # Initialize the Random Forest classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    print(classification_report(y_test, y_pred_rf))
    return y_test, y_pred_rf, rf_model, X_test

#train logistic regression model
def logReg_model(X_train, X_test, y_train, y_test):
    # Initialize the Logistic Regression classifier
    lr_model = LogisticRegression(random_state=1)
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    print(classification_report(y_test, y_pred_lr))
    return y_test, y_pred_lr, lr_model, X_test

In [251]:
# Calculate metrics
def calcMetric(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

def calcMetrics(y_test, y_pred, model, X_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # For binary classifications

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC Score: {roc_auc}")

In [252]:
def lin_svc_pipeline(og_df):
    print("Linear Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred= lin_svc_model(X_train, X_test, y_train, y_test)
    calcMetric(y_test, y_pred)

def svc_pipeline(og_df):
    print("Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = svc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

def rfc_pipeline(og_df):
    print("Random Forest Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = rfc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)
    
def logReg_pipeline(og_df):
    print("Logistic regression Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = logReg_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

In [253]:
# dataframes = [df_hand_int_encoded_NN,df_hand_1hot_encoded_NN,df_hand_int_N_all,df_hand_1hot_N_all]
# dataframe_names = ["hand_int_encoded_NN","hand_1hot_encoded_NN","hand_int_N_all","hand_1hot_N_all"]

dataframes = [df_hand_int_N_all,df_hand_1hot_N_all,df_hand_int_encoded_N_year_bool,df_hand_1hot_encoded_N_year_bool]
dataframe_names = ["hand_int_N_all","hand_1hot_N_all","df_hand_int_encoded_N_year_bool","df_hand_1hot_encoded_N_year_bool"]

In [254]:
i = 0
for df in dataframes:
    print(" \n"+dataframe_names[i])
    lin_svc_pipeline(df)
    print(" \n"+dataframe_names[i])
    rfc_pipeline(df)
    print(" \n"+dataframe_names[i])
    logReg_pipeline(df)
    i+=1

 
hand_int_N_all
Linear Support Vector Classifier
              precision    recall  f1-score   support

           0       0.62      0.63      0.63      3554
           1       0.63      0.63      0.63      3613

    accuracy                           0.63      7167
   macro avg       0.63      0.63      0.63      7167
weighted avg       0.63      0.63      0.63      7167

Accuracy: 0.6274591879447468
Precision: 0.6318881118881119
Recall: 0.6252421810130085
F1 Score: 0.6285475792988314
 
hand_int_N_all
Random Forest Classifier
              precision    recall  f1-score   support

           0       0.69      0.69      0.69      3554
           1       0.70      0.69      0.69      3613

    accuracy                           0.69      7167
   macro avg       0.69      0.69      0.69      7167
weighted avg       0.69      0.69      0.69      7167

Accuracy: 0.6902469652574299
Precision: 0.6950434052086251
Recall: 0.6869637420426239
F1 Score: 0.6909799554565702
ROC-AUC Score: 0.7528587